# Experiment 001: Establish Baseline

Copy the best pre-optimized submission and verify its score.

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import os
import json

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
BASE_TREE = Polygon(zip(TX, TY))

print(f"Tree area: {BASE_TREE.area:.4f}")
print(f"Tree bounds: {BASE_TREE.bounds}")
print(f"Tree width: {BASE_TREE.bounds[2] - BASE_TREE.bounds[0]:.4f}")
print(f"Tree height: {BASE_TREE.bounds[3] - BASE_TREE.bounds[1]:.4f}")

In [ ]:
def parse_value(s):
    """Parse value with 's' prefix for precision"""
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    """Create a tree polygon at position (x, y) with rotation deg"""
    tree = affinity.rotate(BASE_TREE, deg, origin=(0, 0))
    tree = affinity.translate(tree, x, y)
    return tree

def calculate_score(df):
    """Calculate total score for a submission"""
    total_score = 0
    scores_by_n = {}
    
    for n in range(1, 201):
        # Get trees for this N
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        
        if len(n_trees) != n:
            print(f"Warning: N={n} has {len(n_trees)} trees, expected {n}")
            continue
        
        # Create polygons and find bounding box
        all_coords = []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = create_tree_polygon(x, y, deg)
            coords = np.array(poly.exterior.coords)
            all_coords.append(coords)
        
        all_coords = np.vstack(all_coords)
        x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
        y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
        side = max(x_range, y_range)
        
        score_n = side**2 / n
        total_score += score_n
        scores_by_n[n] = {'side': side, 'score': score_n}
    
    return total_score, scores_by_n

print("Score calculation functions defined")

In [ ]:
# Load the pre-optimized submission
preopt_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df = pd.read_csv(preopt_path)
print(f"Loaded {len(df)} rows")
print(df.head(10))

In [ ]:
# Calculate score for pre-optimized submission
print("Calculating score for pre-optimized submission...")
total_score, scores_by_n = calculate_score(df)
print(f"\nTotal Score: {total_score:.6f}")

# Show top 15 contributors
print("\nTop 15 score contributors (by score/n):")
sorted_scores = sorted(scores_by_n.items(), key=lambda x: x[1]['score'], reverse=True)[:15]
for n, data in sorted_scores:
    print(f"N={n:3d}: side={data['side']:.4f}, score={data['score']:.4f}")

In [ ]:
# Copy to submission folder
import shutil
os.makedirs('/home/submission', exist_ok=True)
shutil.copy(preopt_path, '/home/submission/submission.csv')
print("Copied pre-optimized submission to /home/submission/submission.csv")

# Verify copy
df_verify = pd.read_csv('/home/submission/submission.csv')
print(f"Verified: {len(df_verify)} rows")
print(df_verify.head())

In [ ]:
# Save metrics
metrics = {'cv_score': total_score}
os.makedirs('/home/code/experiments/001_baseline', exist_ok=True)
with open('/home/code/experiments/001_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Saved metrics: {metrics}")